In [1]:
from leann.api import LeannBuilder, LeannSearcher
import leann_backend_diskann
# 1. Build index (no embeddings stored!)
builder = LeannBuilder(backend_name="diskann")
builder.add_text("Python is a powerful programming language")
builder.add_text("Machine learning transforms industries")  
builder.add_text("Neural networks process complex data")
builder.add_text("Java is a powerful programming language")
builder.add_text("C++ is a powerful programming language")
builder.add_text("C# is a powerful programming language")
builder.build_index("knowledge.leann")

# 2. Search with real-time embeddings
searcher = LeannSearcher("knowledge.leann")
results = searcher.search("C++ programming languages", top_k=2,recompute_beighbor_embeddings=True)

for result in results:
    print(f"Score: {result['score']:.3f} - {result['text']}")

Initializing leann-backend-diskann...
INFO: Registering backend 'diskann'
INFO: DiskANN backend loaded successfully
INFO: LeannBuilder initialized with 'diskann' backend.


/home/ubuntu/LEANN_clean/leann/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO: Computing embeddings for 6 chunks using 'sentence-transformers/all-mpnet-base-v2'...


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.02it/s]


INFO: Building DiskANN index for 6 vectors with metric Metric.INNER_PRODUCT...
Using Inner Product search, so need to pre-process base data into temp file. Please ensure there is additional (n*(d+1)*4) bytes for storing pre-processed base vectors, apart from the interim indices created by DiskANN and the final index.
Pre-processing base file by adding extra coordinate
✅ DiskANN index built successfully at 'knowledge'
Writing bin: knowledge_disk.index_max_base_norm.bin
bin: #pts = 1, #dims = 1, size = 12B
Finished writing bin.
Time for preprocessing data for inner product: 0.000182 seconds
Reading max_norm_of_base from knowledge_disk.index_max_base_norm.bin
Reading bin file knowledge_disk.index_max_base_norm.bin ...
Opening bin file knowledge_disk.index_max_base_norm.bin... 
Metadata: #pts = 1, #dims = 1...
done.
max_norm_of_base: 1
! Using prepped_base file at knowledge_prepped_base.bin
Starting index build: R=32 L=64 Query RAM budget: 4.02653e+09 Indexing ram budget: 8 T: 8
getting bi

...

INFO: Computing embeddings for 1 chunks using 'sentence-transformers/all-mpnet-base-v2'...


Batches: 100%|██████████| 1/1 [00:00<00:00, 57.79it/s]

INFO: DiskANN ZMQ mode enabled - ensuring embedding server is running
INFO: Starting session-level embedding server as a background process...
INFO: Running command from project root: /home/ubuntu/LEANN_clean/leann
INFO: Server process started with PID: 197427


✅ Embedding server is up and ready for this session.
reserve ratio: 1[EmbeddingServer LOG]: Initializing leann-backend-diskann...
[EmbeddingServer LOG]: WARNING: Could not import DiskANN backend: cannot import name '_diskannpy' from partially initialized module 'packages.leann-backend-diskann.leann_backend_diskann' (most likely due to a circular import) (/home/ubuntu/LEANN_clean/leann/packages/leann-backend-diskann/leann_backend_diskann/__init__.py)
[EmbeddingServer LOG]: INFO: Initializing embedding server thread on port 5555
[EmbeddingServer LOG]: INFO: Using CUDA device
[EmbeddingServer LOG]: INFO: Loading model sentence-transformers/all-mpnet-base-v2
[EmbeddingServer LOG]: INFO: Using FP16 precision with model: sentence-transformers/all-mpnet-base-v2
[EmbeddingServer LOG]: INFO: Loaded 6 demo documents
[EmbeddingServer LOG]: INFO: ZMQ ROUTER server listening on port 5555
[EmbeddingServer LOG]: INFO: Embedding server ready to serve requests
Score: -0.481 - C++ is a powerful programm

[EmbeddingServer LOG]: INFO: ZMQ socket timeout, continuing to listen
[EmbeddingServer LOG]: INFO: ZMQ socket timeout, continuing to listen
[EmbeddingServer LOG]: INFO: ZMQ socket timeout, continuing to listen
[EmbeddingServer LOG]: INFO: ZMQ socket timeout, continuing to listen
[EmbeddingServer LOG]: INFO: ZMQ socket timeout, continuing to listen
[EmbeddingServer LOG]: INFO: ZMQ socket timeout, continuing to listen
